In [9]:
import torch
import torch.nn as nn
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)
np.random.seed(0)

In [5]:
def generate_3D_ellipse(n, r1 = 1, r2 = 0.5):
    t = torch.linspace(-3*torch.pi, 3 * torch.pi, n)
    x = (r1 * torch.cos(t)).view(-1, 1)
    y = (r2 * torch.sin(t)).view(-1, 1)
    z = (0 * torch.sin(t)).view(-1, 1)
    return torch.cat((x, y, z), dim=1).unsqueeze(0)

In [22]:
data = generate_3D_ellipse(6)

In [23]:
def generate_NN_latent_functions(num_samples, xdim=1, zdim=2, lambda_value=1):
    class NN(nn.Module):
        def __init__(self, input_dim, output_dim):
            super(NN, self).__init__()
            self.fc1 = nn.Linear(input_dim, 100)
            self.fc2 = nn.Linear(100, 100)
            self.fc3 = nn.Linear(100, 50)
            self.fc4 = nn.Linear(50, output_dim)
        
        def forward(self, x):
            x = (self.fc1(x))
            x = (self.fc2(x))
            x = (self.fc3(x))
            x = self.fc4(x)
            return x

    #  weight initialization function
    def weights_init_normal(m):
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, mean=0.0, std=1.0)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    #  neural networks
    networks = []
    for _ in range(num_samples):
        net = NN(xdim, zdim)
        net.apply(weights_init_normal)
        networks.append(net)
    return networks

In [24]:
NNs_test = generate_NN_latent_functions(2)
model = NNs_test[0].to(device)

In [25]:
t = torch.linspace(-3*torch.pi, 3*torch.pi, 6).to(device).unsqueeze(1)
print(t)
print(NNs_test[0].to(device)(t))
print(NNs_test[1].to(device)(t))


tensor([[-9.4248],
        [-5.6549],
        [-1.8850],
        [ 1.8850],
        [ 5.6549],
        [ 9.4248]], device='cuda:0')
tensor([[-7546.5513, -4974.4814],
        [-4527.9331, -2984.6890],
        [-1509.3104,  -994.8967],
        [ 1509.3104,   994.8967],
        [ 4527.9331,  2984.6890],
        [ 7546.5513,  4974.4814]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-5151.4287, -2511.2825],
        [-3090.8569, -1506.7711],
        [-1030.2854,  -502.2565],
        [ 1030.2854,   502.2565],
        [ 3090.8569,  1506.7711],
        [ 5151.4287,  2511.2825]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
class H_theta(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(H_theta, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 200),
            nn.ReLU(),
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, output_dim)
        )

    def forward(self, x):
        return self.fc(x)

In [ ]:
H_t = H_theta(input_dim=2, output_dim=2)